# NO_ANN

In [2]:
# --- Imports ---
import os
from pathlib import Path
import numpy as np
import pandas as pd
import pyreadr
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# =========================
# Project root 
# =========================
def detect_project_root():
    env = os.environ.get("CF_CONT_ROOT")
    if env:
        return Path(env).expanduser().resolve()
    try:
        here = Path(__file__).resolve()
    except NameError:
        here = Path.cwd().resolve()
    for p in [here] + list(here.parents):
        if p.name == "CF_Continuous":
            return p
        if (p / "data" / "data_20230504").exists() and (p / "codes").exists():
            return p
    return here

PROJ_ROOT = detect_project_root()

# =========================
# Paths 
# =========================
SPLIT_DIR = PROJ_ROOT / "data" / "train_test_split"
DATA_2ND_STAGE_RDS = PROJ_ROOT / "data" / "data_20230504" / "data_2nd_stage.rds"
EVALL_N_SEQ_RDS    = PROJ_ROOT / "data" / "data_20230504" / "evall_N_seq.rds"

# =========================
# Output 
# =========================
model_tag   = "NO_ANN"
n_fields    = 20               # choose 1, 5, or 10
run_test_id = 1                # run only this test_id
fields_label_map = {1: "one_field", 5: "five_fields", 10: "ten_fields"}
fields_label = fields_label_map.get(n_fields, f"{n_fields}_fields")

RESULTS_BASE = PROJ_ROOT / "results" / "yield_response_function_for_one_iteration"
RESULTS_DIR  = RESULTS_BASE / f"YRF_{model_tag}_{fields_label}"
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# === Model: Simple ANN (NO ANN) ===
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(4, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
    def forward(self, x):
        return self.net(x)

# --- Load data ---
data_2nd_stage = next(iter(pyreadr.read_r(str(DATA_2ND_STAGE_RDS)).values()))
evall_N_seq    = next(iter(pyreadr.read_r(str(EVALL_N_SEQ_RDS)).values()))

# Coerce numerics (include b1, b2 so we can compute true_yield via QP)
for c in ['yield','Nk','plateau','b0','b1','b2','N']:
    if c in data_2nd_stage.columns:
        data_2nd_stage[c] = pd.to_numeric(data_2nd_stage[c], errors='coerce')
for c in ['sim','N','N_tilde']:
    if c in evall_N_seq.columns:
        evall_N_seq[c] = pd.to_numeric(evall_N_seq[c], errors='coerce')

# Clean
data_2nd_stage = data_2nd_stage.dropna(subset=['yield','Nk','plateau','b0','N']).reset_index(drop=True)
evall_N_seq    = evall_N_seq.dropna(subset=['N']).reset_index(drop=True)

# --- Load split CSV and restrict to one test_id ---
split_csv_path = SPLIT_DIR / f"train_test_splits_{n_fields}fields.csv"
splits_df = pd.read_csv(split_csv_path)
splits_df = splits_df[splits_df['test_id'] == run_test_id].iloc[:1].copy()

# Features used for the ANN
feature_cols = ['Nk', 'plateau', 'b0', 'N']

# === Loop (only one iteration) ===
for _, row in tqdm(splits_df.iterrows(), total=len(splits_df), desc="Processing test_id"):
    test_id = int(row['test_id'])
    train_ids = row[[c for c in row.index if c.startswith('train_')]].values

    # ------- Train/val -------
    dataset = data_2nd_stage[data_2nd_stage['sim'].isin(train_ids)].reset_index(drop=True)
    dataset = dataset[['yield'] + feature_cols].copy()

    train_df = dataset.sample(frac=0.8, random_state=0)
    val_df   = dataset.drop(train_df.index)

    X_train = train_df.drop('yield', axis=1)
    y_train = train_df['yield'].to_numpy().reshape(-1, 1)
    X_val   = val_df.drop('yield', axis=1)
    y_val   = val_df['yield'].to_numpy().reshape(-1, 1)

    scaler = StandardScaler().fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_val_scaled   = scaler.transform(X_val)

    X_train_t = torch.tensor(X_train_scaled, dtype=torch.float32, device=device)
    y_train_t = torch.tensor(y_train,       dtype=torch.float32, device=device)
    X_val_t   = torch.tensor(X_val_scaled,  dtype=torch.float32, device=device)
    y_val_t   = torch.tensor(y_val,         dtype=torch.float32, device=device)

    train_ds = TensorDataset(X_train_t, y_train_t)
    train_ld = DataLoader(train_ds, batch_size=512, shuffle=True)

    model = MyModel().to(device)
    criterion = nn.L1Loss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    best_val = float('inf')
    patience = 10
    counter = 0
    max_epochs = 500

    for epoch in range(max_epochs):
        model.train()
        for xb, yb in train_ld:
            optimizer.zero_grad()
            loss = criterion(model(xb), yb)
            loss.backward()
            optimizer.step()

        model.eval()
        with torch.no_grad():
            val_preds = model(X_val_t)
            val_loss = criterion(val_preds, y_val_t).item()

        if val_loss < best_val:
            best_val = val_loss
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                print(f"Early stop at epoch {epoch} (sim {test_id})")
                break

    # ------- Save validation preds -------
    model.eval()
    with torch.no_grad():
        val_out = model(X_val_t).cpu().numpy().flatten()
    pd.DataFrame({'pred': val_out, 'true': y_val.flatten()}).to_csv(
        RESULTS_DIR / f'validation_{test_id}.csv', index=False
    )

    # ------- Yield response function (NO EONR) + TRUE yield (Quadratic-Plateau) -------
    test_df = data_2nd_stage[data_2nd_stage['sim'] == test_id].reset_index(drop=True)

    # ensure we have parameters for true curve
    required_true_cols = ['b0','b1','b2','Nk']
    missing = [c for c in required_true_cols if c not in test_df.columns]
    if missing:
        raise ValueError(f"Missing columns for true yield (QP): {missing}")

    # use these for repeating features (keep plateau for scaler input)
    base_feats = test_df[['Nk', 'plateau', 'b0', 'b1', 'b2']].reset_index(drop=True)

    # N sequence for this sim
    eval_seq = evall_N_seq[evall_N_seq['sim'] == test_id].reset_index(drop=True)
    if eval_seq.empty:
        (RESULTS_DIR / f'yield_response_{test_id}_EMPTY_EVAL_SEQ.csv').write_text(
            "No eval N sequence found for this sim\n"
        )
        continue

    Nseq = eval_seq['N'].to_numpy()
    L = len(Nseq)

    #  identifiers to carry through
    id_cols = [c for c in ['aunit_id', 'cell_id', 'field_id'] if c in test_df.columns]

    all_preds = []
    with torch.no_grad():
        for i in range(len(base_feats)):
            base = base_feats.iloc[[i]][['Nk','plateau','b0']]  # features used by the ANN
            repeated = pd.concat([base] * L, ignore_index=True)
            full_feat = pd.concat([repeated, eval_seq[['N']]], axis=1)
            full_feat = full_feat[['Nk', 'plateau', 'b0', 'N']]

            # model predictions
            X_feat = torch.tensor(scaler.transform(full_feat), dtype=torch.float32, device=device)
            y_hat  = model(X_feat).cpu().numpy().reshape(-1)

            # TRUE yield via Quadratic-Plateau:
            # y(N) = b0 + b1*N + b2*N^2  (for N < Nk), else y = b0 + b1*Nk + b2*Nk^2
            b0_i = float(base_feats.loc[i, 'b0'])
            b1_i = float(base_feats.loc[i, 'b1'])
            b2_i = float(base_feats.loc[i, 'b2'])
            Nk_i = float(base_feats.loc[i, 'Nk'])

            quad_vals    = b0_i + b1_i * Nseq + b2_i * (Nseq**2)
            plateau_val  = b0_i + b1_i * Nk_i + b2_i * (Nk_i**2)
            true_y       = np.where(Nseq < Nk_i, quad_vals, plateau_val)

            row = {
                'sim':         [test_id] * L,
                'row_id':      [i] * L,
                'N':           Nseq,
                'pred_yield':  y_hat,
                'true_yield':  true_y,   # NEW column
            }
            # keep N_tilde if present in eval_seq
            if 'N_tilde' in eval_seq.columns:
                row['N_tilde'] = eval_seq['N_tilde'].to_numpy()
            # carry IDs
            for col in id_cols:
                row[col] = [test_df.loc[i, col]] * L

            all_preds.append(pd.DataFrame(row))

    df_out = pd.concat(all_preds, ignore_index=True)
    df_out.to_csv(RESULTS_DIR / f'yield_response_{test_id}.csv', index=False)


Processing test_id:   0%|          | 0/1 [00:00<?, ?it/s]

Early stop at epoch 106 (sim 1)


Processing test_id: 100%|██████████| 1/1 [00:12<00:00, 12.60s/it]


# NO_RF

In [4]:
# --- Imports ---
import os
from pathlib import Path
import numpy as np
import pandas as pd
import pyreadr
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RepeatedKFold, GridSearchCV

# =========================
# Project root 
# =========================
def detect_project_root():
    env = os.environ.get("CF_CONT_ROOT")
    if env:
        return Path(env).expanduser().resolve()
    try:
        here = Path(__file__).resolve()
    except NameError:  # e.g., notebooks
        here = Path.cwd().resolve()
    for p in [here] + list(here.parents):
        if p.name == "CF_Continuous":
            return p
        if (p / "data" / "data_20230504").exists() and (p / "codes").exists():
            return p
    return here

PROJ_ROOT = detect_project_root()

# =========================
# Paths 
# =========================
SPLIT_DIR          = PROJ_ROOT / "data" / "train_test_split"
DATA_2ND_STAGE_RDS = PROJ_ROOT / "data" / "data_20230504" / "data_2nd_stage.rds"
EVALL_N_SEQ_RDS    = PROJ_ROOT / "data" / "data_20230504" / "evall_N_seq.rds"

# Output 
model_tag   = "NO_RF"
n_fields    = 20               # choose 1, 5, or 10
run_test_id = 1                # run only this test_id
fields_label_map = {1: "one_field", 5: "five_fields", 10: "ten_fields"}
fields_label = fields_label_map.get(n_fields, f"{n_fields}_fields")

RESULTS_BASE = PROJ_ROOT / "results" / "yield_response_function_for_one_iteration"
RESULTS_DIR  = RESULTS_BASE / f"YRF_{model_tag}_{fields_label}"
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

# === Load data from .rds ===
data_2nd_stage = next(iter(pyreadr.read_r(str(DATA_2ND_STAGE_RDS)).values()))
evall_N_seq    = next(iter(pyreadr.read_r(str(EVALL_N_SEQ_RDS)).values()))

# Make numerics (add b1, b2 so we can compute true_yield)
for c in ['yield', 'Nk', 'plateau', 'b0', 'b1', 'b2', 'N']:
    if c in data_2nd_stage.columns:
        data_2nd_stage[c] = pd.to_numeric(data_2nd_stage[c], errors='coerce')
for c in ['sim', 'N', 'N_tilde']:
    if c in evall_N_seq.columns:
        evall_N_seq[c] = pd.to_numeric(evall_N_seq[c], errors='coerce')

# Drop rows with NaNs in key columns for training
data_2nd_stage = data_2nd_stage.dropna(subset=['yield','Nk','plateau','b0','N']).reset_index(drop=True)
evall_N_seq    = evall_N_seq.dropna(subset=['N']).reset_index(drop=True)

# === Load split CSV and restrict to one test_id ===
split_csv_path = SPLIT_DIR / f"train_test_splits_{n_fields}fields.csv"
splits_df = pd.read_csv(split_csv_path)
splits_df = splits_df[splits_df['test_id'] == run_test_id].iloc[:1].copy()

# === Features for training ===
feature_cols = ['Nk', 'plateau', 'b0', 'N']

# === Loop (only one iteration) ===
for _, row in tqdm(splits_df.iterrows(), total=len(splits_df), desc="Processing test_id"):
    test_id = int(row['test_id'])
    train_ids = row[[c for c in row.index if c.startswith('train_')]].values

    # -------------------------
    # train/val data
    # -------------------------
    dataset = data_2nd_stage[data_2nd_stage['sim'].isin(train_ids)].reset_index(drop=True)
    dataset = dataset[['yield'] + feature_cols].copy()

    # Random 80/20 split
    train_df = dataset.sample(frac=0.8, random_state=0)
    val_df   = dataset.drop(train_df.index)

    X_train = train_df.drop('yield', axis=1)
    y_train = train_df['yield']
    X_val   = val_df.drop('yield', axis=1)
    y_val   = val_df['yield']

    scaler = StandardScaler().fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_val_scaled   = scaler.transform(X_val)

    # -------------------------
    # Random Forest with CV
    # -------------------------
    param_grid = {
        'max_depth':    [3, 5],
        'n_estimators': [50, 250, 500, 1000],
        'max_features': [1, 2, 3],
    }
    rf = RandomForestRegressor(random_state=777)
    cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=777)
    grid = GridSearchCV(rf, param_grid, cv=cv, n_jobs=-1, scoring='neg_mean_squared_error')
    grid.fit(X_train_scaled, y_train)
    model = grid.best_estimator_

    # -------------------------
    # Save validation predictions
    # -------------------------
    val_preds = model.predict(X_val_scaled)
    pd.DataFrame({'pred': val_preds, 'true': y_val.values}).to_csv(
        RESULTS_DIR / f'validation_{test_id}.csv', index=False
    )

    # -------------------------
    # Yield response function (no EONR) + TRUE yield (Quadratic-Plateau)
    # -------------------------
    test_df  = data_2nd_stage[data_2nd_stage['sim'] == test_id].reset_index(drop=True)

    # Ensure parameters are available for the true curve
    req_true = ['b0','b1','b2','Nk']
    missing  = [c for c in req_true if c not in test_df.columns]
    if missing:
        raise ValueError(f"Missing columns for true yield (QP): {missing}")

    features = test_df[['Nk', 'plateau', 'b0', 'b1', 'b2']].reset_index(drop=True)

    # N sequence for THIS test_id
    eval_seq = evall_N_seq[evall_N_seq['sim'] == test_id].reset_index(drop=True)
    if eval_seq.empty:
        (RESULTS_DIR / f'yield_response_{test_id}_EMPTY_EVAL_SEQ.csv').write_text(
            "No eval N sequence found for this sim\n"
        )
        continue

    Nseq = eval_seq['N'].to_numpy()
    L = len(Nseq)

    #  IDs to carry through
    id_cols = [c for c in ['aunit_id', 'cell_id', 'field_id'] if c in test_df.columns]

    all_preds = []
    for i in range(len(features)):
        base = features.iloc[[i]][['Nk','plateau','b0']]          # features used by RF
        repeated = pd.concat([base] * L, ignore_index=True)
        full_feat = pd.concat([repeated, eval_seq[['N']]], axis=1)
        full_feat = full_feat[['Nk', 'plateau', 'b0', 'N']]

        # model predictions
        X_feat = scaler.transform(full_feat)
        preds  = model.predict(X_feat)

        # TRUE yield via quadratic-plateau:
        # y(N) = b0 + b1*N + b2*N^2  if N < Nk
        #      = b0 + b1*Nk + b2*Nk^2 otherwise
        b0_i = float(features.loc[i, 'b0'])
        b1_i = float(features.loc[i, 'b1'])
        b2_i = float(features.loc[i, 'b2'])
        Nk_i = float(features.loc[i, 'Nk'])

        quad_vals   = b0_i + b1_i * Nseq + b2_i * (Nseq**2)
        plateau_val = b0_i + b1_i * Nk_i + b2_i * (Nk_i**2)
        true_y      = np.where(Nseq < Nk_i, quad_vals, plateau_val)

        row_dict = {
            'sim':         [test_id] * L,
            'row_id':      [i] * L,
            'N':           Nseq,
            'pred_yield':  preds,
            'true_yield':  true_y
        }
        # keep N_tilde if present (useful for other models)
        if 'N_tilde' in eval_seq.columns:
            row_dict['N_tilde'] = eval_seq['N_tilde'].to_numpy()
        for col in id_cols:
            row_dict[col] = [test_df.loc[i, col]] * L

        all_preds.append(pd.DataFrame(row_dict))

    df_out = pd.concat(all_preds, ignore_index=True)
    df_out.to_csv(RESULTS_DIR / f'yield_response_{test_id}.csv', index=False)

# --- Note ---
# If test_df has 1,440 rows and N sequence has 100 values → output CSV has 144,000 rows.



Processing test_id: 100%|██████████| 1/1 [04:37<00:00, 277.07s/it]


# SO_ANN

In [6]:
# --- Imports ---
import os
from pathlib import Path
import numpy as np
import pandas as pd
import pyreadr
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# =========================
# Project root 
# =========================
def detect_project_root():
    env = os.environ.get("CF_CONT_ROOT")
    if env:
        return Path(env).expanduser().resolve()
    try:
        here = Path(__file__).resolve()
    except NameError:
        here = Path.cwd().resolve()
    for p in [here] + list(here.parents):
        if p.name == "CF_Continuous":
            return p
        if (p / "data" / "data_20230504").exists() and (p / "codes").exists():
            return p
    return here

PROJ_ROOT = detect_project_root()

# =========================
# Paths
# =========================
SPLIT_DIR = PROJ_ROOT / "data" / "train_test_split"
DATA_2ND_STAGE_RDS = PROJ_ROOT / "data" / "data_20230504" / "data_2nd_stage.rds"
EVALL_N_SEQ_RDS    = PROJ_ROOT / "data" / "data_20230504" / "evall_N_seq.rds"

# =========================
# Output 
# =========================
model_tag   = "SO_ANN"
n_fields    = 20              # choose 1, 5, or 10
run_test_id = 1                # run only this test_id
fields_label_map = {1: "one_field", 5: "five_fields", 10: "ten_fields"}
fields_label = fields_label_map.get(n_fields, f"{n_fields}_fields")

RESULTS_BASE = PROJ_ROOT / "results" / "yield_response_function_for_one_iteration"
RESULTS_DIR  = RESULTS_BASE / f"YRF_{model_tag}_{fields_label}"
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# === Model  ===
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(4, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
    def forward(self, x):
        return self.net(x)

# === Load data from .rds ===
data_2nd_stage = next(iter(pyreadr.read_r(str(DATA_2ND_STAGE_RDS)).values()))
evall_N_seq    = next(iter(pyreadr.read_r(str(EVALL_N_SEQ_RDS)).values()))

# Make numerics (include b1, b2 to compute true_yield)
for c in ['y_tilde', 'Nk', 'plateau', 'b0', 'b1', 'b2', 'N']:
    if c in data_2nd_stage.columns:
        data_2nd_stage[c] = pd.to_numeric(data_2nd_stage[c], errors='coerce')
for c in ['sim', 'N', 'N_tilde']:
    if c in evall_N_seq.columns:
        evall_N_seq[c] = pd.to_numeric(evall_N_seq[c], errors='coerce')

# Drop rows with NaNs for training
data_2nd_stage = data_2nd_stage.dropna(subset=['y_tilde','Nk','plateau','b0','N']).reset_index(drop=True)
evall_N_seq    = evall_N_seq.dropna(subset=['N']).reset_index(drop=True)

# === Load split CSV and restrict to one test_id ===
split_csv_path = SPLIT_DIR / f"train_test_splits_{n_fields}fields.csv"
splits_df = pd.read_csv(split_csv_path)
splits_df = splits_df[splits_df['test_id'] == run_test_id].iloc[:1].copy()

# === Features for training ===
feature_cols = ['Nk', 'plateau', 'b0', 'N']

# === Loop (only one iteration) ===
for _, row in tqdm(splits_df.iterrows(), total=len(splits_df), desc="Processing test_id"):
    test_id = int(row['test_id'])
    train_ids = row[[c for c in row.index if c.startswith('train_')]].values

    # -------------------------
    # train/val data
    # -------------------------
    dataset = data_2nd_stage[data_2nd_stage['sim'].isin(train_ids)].reset_index(drop=True)
    dataset = dataset[['y_tilde'] + feature_cols].copy()

    # Random 80/20 split
    train_df = dataset.sample(frac=0.8, random_state=0)
    val_df   = dataset.drop(train_df.index)

    X_train = train_df.drop('y_tilde', axis=1)
    y_train = train_df['y_tilde'].to_numpy().reshape(-1, 1)
    X_val   = val_df.drop('y_tilde', axis=1)
    y_val   = val_df['y_tilde'].to_numpy().reshape(-1, 1)

    scaler = StandardScaler().fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_val_scaled   = scaler.transform(X_val)

    X_train_t = torch.tensor(X_train_scaled, dtype=torch.float32, device=device)
    y_train_t = torch.tensor(y_train,       dtype=torch.float32, device=device)
    X_val_t   = torch.tensor(X_val_scaled,  dtype=torch.float32, device=device)
    y_val_t   = torch.tensor(y_val,         dtype=torch.float32, device=device)

    train_ds = TensorDataset(X_train_t, y_train_t)
    train_ld = DataLoader(train_ds, batch_size=512, shuffle=True)

    model = MyModel().to(device)
    criterion = nn.L1Loss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    best_val = float('inf')
    patience = 10
    counter = 0
    max_epochs = 500

    for epoch in range(max_epochs):
        model.train()
        for xb, yb in train_ld:
            optimizer.zero_grad()
            preds = model(xb)
            loss = criterion(preds, yb)
            loss.backward()
            optimizer.step()

        model.eval()
        with torch.no_grad():
            val_preds = model(X_val_t)
            val_loss = criterion(val_preds, y_val_t).item()

        if val_loss < best_val:
            best_val = val_loss
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                print(f"Early stop at epoch {epoch} (sim {test_id})")
                break

    # -------------------------
    # Save validation predictions
    # -------------------------
    model.eval()
    with torch.no_grad():
        val_out = model(X_val_t).cpu().numpy().flatten()
    pd.DataFrame({'pred': val_out, 'true': y_val.flatten()}).to_csv(
        RESULTS_DIR / f'validation_{test_id}.csv', index=False
    )

    # -------------------------
    # Yield response function (NO EONR) + TRUE yield (Quadratic-Plateau)
    # -------------------------
    test_df    = data_2nd_stage[data_2nd_stage['sim'] == test_id].reset_index(drop=True)
    base_feats = test_df[['Nk', 'plateau', 'b0']].reset_index(drop=True)

    # Ensure parameters exist for TRUE curve
    req_true = ['b0','b1','b2','Nk']
    missing  = [c for c in req_true if c not in test_df.columns]
    if missing:
        raise ValueError(f"Missing columns for true yield (QP): {missing}")

    # N sequence for THIS test_id
    eval_seq = evall_N_seq[evall_N_seq['sim'] == test_id].reset_index(drop=True)
    if eval_seq.empty:
        (RESULTS_DIR / f'yield_response_{test_id}_EMPTY_EVAL_SEQ.csv').write_text(
            "No eval N sequence found for this sim\n"
        )
        continue

    Nseq = eval_seq['N'].to_numpy()
    L = len(Nseq)

    # identifiers 
    id_cols = [c for c in ['aunit_id', 'cell_id', 'field_id'] if c in test_df.columns]

    all_preds = []
    with torch.no_grad():
        for i in range(len(base_feats)):
            base = base_feats.iloc[[i]]                              # 1×3 (Nk, plateau, b0)
            repeated = pd.concat([base] * L, ignore_index=True)      # L×3
            full_feat = pd.concat([repeated, eval_seq[['N']]], axis=1)  # L×4
            full_feat = full_feat[['Nk', 'plateau', 'b0', 'N']]      

            X_feat = torch.tensor(scaler.transform(full_feat), dtype=torch.float32, device=device)
            y_hat  = model(X_feat).cpu().numpy().reshape(-1)

            # --- TRUE yield via quadratic-plateau ---
            b0_i = float(test_df.loc[i, 'b0'])
            b1_i = float(test_df.loc[i, 'b1'])
            b2_i = float(test_df.loc[i, 'b2'])
            Nk_i = float(test_df.loc[i, 'Nk'])

            quad_vals   = b0_i + b1_i * Nseq + b2_i * (Nseq**2)
            plateau_val = b0_i + b1_i * Nk_i + b2_i * (Nk_i**2)
            true_y      = np.where(Nseq < Nk_i, quad_vals, plateau_val)

            row_dict = {
                'sim':         [test_id] * L,
                'row_id':      [i] * L,
                'N':           Nseq,
                'pred_yield':  y_hat,
                'true_yield':  true_y
            }
            # keep N_tilde if present (can be handy for scaled-N models)
            if 'N_tilde' in eval_seq.columns:
                row_dict['N_tilde'] = eval_seq['N_tilde'].to_numpy()
            for col in id_cols:
                row_dict[col] = [test_df.loc[i, col]] * L

            all_preds.append(pd.DataFrame(row_dict))

    df_out = pd.concat(all_preds, ignore_index=True)
    df_out.to_csv(RESULTS_DIR / f'yield_response_{test_id}.csv', index=False)


Processing test_id:   0%|          | 0/1 [00:00<?, ?it/s]

Early stop at epoch 78 (sim 1)


Processing test_id: 100%|██████████| 1/1 [00:09<00:00,  9.13s/it]


# DO_ANN

In [8]:
# --- Imports ---
import os
from pathlib import Path
import numpy as np
import pandas as pd
import pyreadr
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.optim as optim

# =========================
# Project root 
# =========================
def detect_project_root():
    env = os.environ.get("CF_CONT_ROOT")
    if env:
        return Path(env).expanduser().resolve()
    try:
        here = Path(__file__).resolve()
    except NameError:
        here = Path.cwd().resolve()
    for p in [here] + list(here.parents):
        if p.name == "CF_Continuous":
            return p
        if (p / "data" / "data_20230504").exists() and (p / "codes").exists():
            return p
    return here

PROJ_ROOT = detect_project_root()

# =========================
# Paths 
# =========================
SPLIT_DIR = PROJ_ROOT / "data" / "train_test_split"
DATA_2ND_STAGE_RDS = PROJ_ROOT / "data" / "data_20230504" / "data_2nd_stage.rds"
EVALL_N_SEQ_RDS    = PROJ_ROOT / "data" / "data_20230504" / "evall_N_seq.rds"

# =========================
# Output 
# =========================
model_tag   = "DO_ANN"
n_fields    = 20                 # choose 1, 5, or 10
run_test_id = 1                 # run only this test_id
fields_label_map = {1: "one_field", 5: "five_fields", 10: "ten_fields"}
fields_label = fields_label_map.get(n_fields, f"{n_fields}_fields")

RESULTS_BASE = PROJ_ROOT / "results" / "yield_response_function_for_one_iteration"
RESULTS_DIR  = RESULTS_BASE / f"YRF_{model_tag}_{fields_label}"
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# === Model  ===
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.branch = nn.Sequential(
            nn.Linear(3, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
    def forward(self, inp):
        x = inp[:, :3]   # Nk, plateau, b0
        n = inp[:, 3:4]  # N_tilde (scaled)
        return self.branch(x) * n

# === Load data from .rds ===
data_2nd_stage = next(iter(pyreadr.read_r(str(DATA_2ND_STAGE_RDS)).values()))
evall_N_seq    = next(iter(pyreadr.read_r(str(EVALL_N_SEQ_RDS)).values()))

# Make numerics (include b1, b2 so we can compute true_yield)
for c in ['y_tilde', 'Nk', 'plateau', 'b0', 'b1', 'b2', 'N_tilde', 'N']:
    if c in data_2nd_stage.columns:
        data_2nd_stage[c] = pd.to_numeric(data_2nd_stage[c], errors='coerce')
for c in ['sim', 'N_tilde', 'N']:
    if c in evall_N_seq.columns:
        evall_N_seq[c] = pd.to_numeric(evall_N_seq[c], errors='coerce')

# Clean
data_2nd_stage = data_2nd_stage.dropna(subset=['y_tilde','Nk','plateau','b0','N_tilde']).reset_index(drop=True)
# Need at least N_tilde in eval sequence
need_eval_cols = ['sim', 'N_tilde']
if not set(need_eval_cols).issubset(evall_N_seq.columns):
    raise ValueError("evall_N_seq must contain columns: 'sim' and 'N_tilde'")
evall_N_seq = evall_N_seq.dropna(subset=['N_tilde']).reset_index(drop=True)

# === Load split CSV and restrict to one test_id ===
split_csv_path = SPLIT_DIR / f"train_test_splits_{n_fields}fields.csv"
splits_df = pd.read_csv(split_csv_path)
splits_df = splits_df[splits_df['test_id'] == run_test_id].iloc[:1].copy()

# === Features for training ===
feature_cols = ['Nk', 'plateau', 'b0', 'N_tilde']

# === Loop (only one iteration) ===
for _, row in tqdm(splits_df.iterrows(), total=len(splits_df), desc="Processing test_id"):
    test_id = int(row['test_id'])
    train_ids = row[[c for c in row.index if c.startswith('train_')]].values

    # -------------------------
    # train/val data
    # -------------------------
    dataset = data_2nd_stage[data_2nd_stage['sim'].isin(train_ids)].reset_index(drop=True)
    dataset = dataset[['y_tilde'] + feature_cols].copy()

    # Random 80/20 split (reproducible)
    train_df = dataset.sample(frac=0.8, random_state=0)
    val_df   = dataset.drop(train_df.index)

    X_train = train_df[feature_cols]
    y_train = train_df['y_tilde'].to_numpy().reshape(-1, 1)
    X_val   = val_df[feature_cols]
    y_val   = val_df['y_tilde'].to_numpy().reshape(-1, 1)

    scaler = StandardScaler().fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_val_scaled   = scaler.transform(X_val)

    X_train_t = torch.tensor(X_train_scaled, dtype=torch.float32, device=device)
    y_train_t = torch.tensor(y_train,       dtype=torch.float32, device=device)
    X_val_t   = torch.tensor(X_val_scaled,  dtype=torch.float32, device=device)
    y_val_t   = torch.tensor(y_val,         dtype=torch.float32, device=device)

    model = MyModel().to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.L1Loss()

    best_val_loss = float('inf')
    patience = 10
    counter = 0
    max_epochs = 500

    for epoch in range(max_epochs):
        model.train()
        # manual mini-batch
        bs = 512
        for i in range(0, len(X_train_t), bs):
            xb = X_train_t[i:i+bs]
            yb = y_train_t[i:i+bs]
            optimizer.zero_grad()
            loss = criterion(model(xb), yb)
            loss.backward()
            optimizer.step()

        model.eval()
        with torch.no_grad():
            val_loss = criterion(model(X_val_t), y_val_t).item()

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                print(f"Early stop at epoch {epoch} (sim {test_id})")
                break

    # -------------------------
    # Save validation predictions
    # -------------------------
    model.eval()
    with torch.no_grad():
        val_preds = model(X_val_t).cpu().numpy().flatten()
    pd.DataFrame({'pred': val_preds, 'true': y_val.flatten()}).to_csv(
        RESULTS_DIR / f'validation_{test_id}.csv', index=False
    )

    # -------------------------
    # Yield response function (NO EONR) + TRUE yield (Quadratic-Plateau)
    # -------------------------
    test_df   = data_2nd_stage[data_2nd_stage['sim'] == test_id].reset_index(drop=True)
    base_feats = test_df[['Nk', 'plateau', 'b0']].reset_index(drop=True)

    # Ensure parameters exist for TRUE curve
    req_true = ['b0','b1','b2','Nk']
    missing  = [c for c in req_true if c not in test_df.columns]
    if missing:
        raise ValueError(f"Missing columns for true yield (QP): {missing}")

    # N sequence (N_tilde) for THIS test_id
    eval_seq = evall_N_seq[evall_N_seq['sim'] == test_id].reset_index(drop=True)
    if eval_seq.empty:
        (RESULTS_DIR / f'yield_response_{test_id}_EMPTY_EVAL_SEQ.csv').write_text(
            "No eval N_tilde sequence found for this sim\n"
        )
        continue

    Nt_seq = eval_seq['N_tilde'].to_numpy()
    # Prefer raw N for the true curve if available; else fall back to N_tilde.
    if 'N' in eval_seq.columns and eval_seq['N'].notna().any():
        N_for_true = eval_seq['N'].to_numpy()
    else:
        N_for_true = Nt_seq  # fallback if raw N not provided
    L = len(Nt_seq)

    # IDs to carry through
    id_cols = [c for c in ['aunit_id', 'cell_id', 'field_id'] if c in test_df.columns]

    all_preds = []
    with torch.no_grad():
        for i in range(len(base_feats)):
            base = base_feats.iloc[[i]]                             # 1×3
            repeated = pd.concat([base] * L, ignore_index=True)     # L×3
            full_feat = pd.concat([repeated, eval_seq[['N_tilde']]], axis=1)  # L×4
            full_feat = full_feat[['Nk', 'plateau', 'b0', 'N_tilde']]         

            X_feat = torch.tensor(scaler.transform(full_feat), dtype=torch.float32, device=device)
            y_hat  = model(X_feat).cpu().numpy().reshape(-1)

            # --- TRUE yield via quadratic-plateau using N_for_true ---
            b0_i = float(test_df.loc[i, 'b0'])
            b1_i = float(test_df.loc[i, 'b1'])
            b2_i = float(test_df.loc[i, 'b2'])
            Nk_i = float(test_df.loc[i, 'Nk'])

            quad_vals   = b0_i + b1_i * N_for_true + b2_i * (N_for_true**2)
            plateau_val = b0_i + b1_i * Nk_i       + b2_i * (Nk_i**2)
            true_y      = np.where(N_for_true < Nk_i, quad_vals, plateau_val)

            row = {
                'sim':         [test_id] * L,
                'row_id':      [i] * L,
                'N_tilde':     Nt_seq,
                'pred_yield':  y_hat,
                'true_yield':  true_y
            }
            if 'N' in eval_seq.columns:
                row['N'] = eval_seq['N'].to_numpy()
            for col in id_cols:
                row[col] = [test_df.loc[i, col]] * L

            all_preds.append(pd.DataFrame(row))

    df_out = pd.concat(all_preds, ignore_index=True)
    df_out.to_csv(RESULTS_DIR / f'yield_response_{test_id}.csv', index=False)


Processing test_id:   0%|          | 0/1 [00:00<?, ?it/s]

Early stop at epoch 172 (sim 1)


Processing test_id: 100%|██████████| 1/1 [00:06<00:00,  6.22s/it]


# SO_RF

In [10]:
# --- Imports ---
import os
from pathlib import Path
import numpy as np
import pandas as pd
import pyreadr
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RepeatedKFold, GridSearchCV

# =========================
# Project root 
# =========================
def detect_project_root():
    env = os.environ.get("CF_CONT_ROOT")
    if env:
        return Path(env).expanduser().resolve()
    try:
        here = Path(__file__).resolve()
    except NameError:  # e.g., notebooks
        here = Path.cwd().resolve()
    for p in [here] + list(here.parents):
        if p.name == "CF_Continuous":
            return p
        if (p / "data" / "data_20230504").exists() and (p / "codes").exists():
            return p
    return here

PROJ_ROOT = detect_project_root()

# =========================
# Paths 
# =========================
SPLIT_DIR = PROJ_ROOT / "data" / "train_test_split"
DATA_2ND_STAGE_RDS = PROJ_ROOT / "data" / "data_20230504" / "data_2nd_stage.rds"
EVALL_N_SEQ_RDS    = PROJ_ROOT / "data" / "data_20230504" / "evall_N_seq.rds"

# =========================
# Output 
# =========================
model_tag   = "SO_RF"
n_fields    = 20                 # choose 1, 5, or 10
run_test_id = 1                 # run only this test_id
fields_label_map = {1: "one_field", 5: "five_fields", 10: "ten_fields"}
fields_label = fields_label_map.get(n_fields, f"{n_fields}_fields")

RESULTS_BASE = PROJ_ROOT / "results" / "yield_response_function_for_one_iteration"
RESULTS_DIR  = RESULTS_BASE / f"YRF_{model_tag}_{fields_label}"
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

# === Load data from .rds ===
data_2nd_stage = next(iter(pyreadr.read_r(str(DATA_2ND_STAGE_RDS)).values()))
evall_N_seq    = next(iter(pyreadr.read_r(str(EVALL_N_SEQ_RDS)).values()))

# Make numerics (include b1, b2 so we can compute true_yield)
for c in ['y_tilde','Nk','plateau','b0','b1','b2','N']:
    if c in data_2nd_stage.columns:
        data_2nd_stage[c] = pd.to_numeric(data_2nd_stage[c], errors='coerce')
for c in ['sim','N','N_tilde']:
    if c in evall_N_seq.columns:
        evall_N_seq[c] = pd.to_numeric(evall_N_seq[c], errors='coerce')

# Drop rows with NaNs 
data_2nd_stage = data_2nd_stage.dropna(subset=['y_tilde','Nk','plateau','b0','N']).reset_index(drop=True)
evall_N_seq    = evall_N_seq.dropna(subset=['N']).reset_index(drop=True)

# === Load split CSV and restrict to one test_id ===
split_csv_path = SPLIT_DIR / f"train_test_splits_{n_fields}fields.csv"
splits_df = pd.read_csv(split_csv_path)
splits_df = splits_df[splits_df['test_id'] == run_test_id].iloc[:1].copy()

# === Features for training/prediction ===
feature_cols = ['Nk', 'plateau', 'b0', 'N']

# === Loop (only one iteration) ===
for _, row in tqdm(splits_df.iterrows(), total=len(splits_df), desc="Processing test_id"):
    test_id = int(row['test_id'])
    train_ids = row[[c for c in row.index if c.startswith('train_')]].values

    # -------------------------
    # Train / validation data
    # -------------------------
    dataset = data_2nd_stage[data_2nd_stage['sim'].isin(train_ids)].reset_index(drop=True)
    dataset = dataset[['y_tilde'] + feature_cols].copy()

    train_df = dataset.sample(frac=0.8, random_state=0)
    val_df   = dataset.drop(train_df.index)

    X_train = train_df.drop('y_tilde', axis=1)
    y_train = train_df['y_tilde']
    X_val   = val_df.drop('y_tilde', axis=1)
    y_val   = val_df['y_tilde']

    scaler = StandardScaler().fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_val_scaled   = scaler.transform(X_val)

    # -------------------------
    # Random Forest with CV
    # -------------------------
    param_grid = {
        'max_depth':    [3, 5],
        'n_estimators': [50, 250, 500, 1000],
        'max_features': [1, 2, 3],
    }
    rf = RandomForestRegressor(random_state=777)
    cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=777)
    grid = GridSearchCV(rf, param_grid, cv=cv, n_jobs=-1, scoring='neg_mean_squared_error')
    grid.fit(X_train_scaled, y_train)
    model = grid.best_estimator_

    # -------------------------
    # Save validation predictions
    # -------------------------
    val_preds = model.predict(X_val_scaled)
    pd.DataFrame({'pred': val_preds, 'true': y_val.values}).to_csv(
        RESULTS_DIR / f'validation_{test_id}.csv', index=False
    )

    # -------------------------
    # Yield response function (NO EONR) + TRUE yield (Quadratic-Plateau)
    # -------------------------
    test_df    = data_2nd_stage[data_2nd_stage['sim'] == test_id].reset_index(drop=True)
    base_feats = test_df[['Nk', 'plateau', 'b0']].reset_index(drop=True)

    # Ensure parameters exist for TRUE curve
    req_true = ['b0','b1','b2','Nk']
    missing  = [c for c in req_true if c not in test_df.columns]
    if missing:
        raise ValueError(f"Missing columns for true yield (QP): {missing}")

    # N sequence for THIS test_id
    eval_seq = evall_N_seq[evall_N_seq['sim'] == test_id].reset_index(drop=True)
    if eval_seq.empty:
        (RESULTS_DIR / f'yield_response_{test_id}_EMPTY_EVAL_SEQ.csv').write_text(
            "No eval N sequence found for this sim\n"
        )
        continue

    Nseq = eval_seq['N'].to_numpy()
    L = len(Nseq)

    # carry optional identifiers
    id_cols = [c for c in ['aunit_id', 'cell_id', 'field_id'] if c in test_df.columns]

    all_preds = []
    for i in range(len(base_feats)):
        base = base_feats.iloc[[i]]                               # 1×3
        repeated = pd.concat([base] * L, ignore_index=True)       # L×3
        full_feat = pd.concat([repeated, eval_seq[['N']]], axis=1)  # L×4
        full_feat = full_feat[['Nk', 'plateau', 'b0', 'N']]      

        X_feat = scaler.transform(full_feat)
        preds  = model.predict(X_feat)

        # --- TRUE yield via quadratic-plateau ---
        b0_i = float(test_df.loc[i, 'b0'])
        b1_i = float(test_df.loc[i, 'b1'])
        b2_i = float(test_df.loc[i, 'b2'])
        Nk_i = float(test_df.loc[i, 'Nk'])

        quad_vals   = b0_i + b1_i * Nseq + b2_i * (Nseq**2)
        plateau_val = b0_i + b1_i * Nk_i + b2_i * (Nk_i**2)
        true_y      = np.where(Nseq < Nk_i, quad_vals, plateau_val)

        row = {
            'sim':         [test_id] * L,
            'row_id':      [i] * L,
            'N':           Nseq,
            'pred_yield':  preds,
            'true_yield':  true_y,
        }
        # keep N_tilde if present (harmless, sometimes available in evall_N_seq)
        if 'N_tilde' in eval_seq.columns:
            row['N_tilde'] = eval_seq['N_tilde'].to_numpy()
        for col in id_cols:
            row[col] = [test_df.loc[i, col]] * L

        all_preds.append(pd.DataFrame(row))

    df_out = pd.concat(all_preds, ignore_index=True)
    df_out.to_csv(RESULTS_DIR / f'yield_response_{test_id}.csv', index=False)

# --- Notes ---
# • Trains on ['Nk','plateau','b0','N'] to predict y_tilde.
# • Output rows = (# test rows for sim) × (length of N sequence for sim).
# • true_yield computed as quadratic-plateau:
#     y = b0 + b1*N + b2*N^2  if N < Nk
#     y = b0 + b1*Nk + b2*Nk^2 otherwise


Processing test_id: 100%|██████████| 1/1 [05:04<00:00, 304.23s/it]


# CF

In [ ]:
# === CF Yield Response Function (mirrors SO_ANN YRF writer) ===
import os
from pathlib import Path
import numpy as np
import pandas as pd
import pyreadr
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler

# === Causal Forest runner (your wrapper) ===
import sys
try:
    this_dir = Path(__file__).parent
except NameError:
    this_dir = Path.cwd()
sys.path.append(str(this_dir.parent))
from run_CF_c import run_CF_c_py

# =============================================================
# Project root detection (same pattern you use)
# =============================================================
def detect_project_root():
    env = os.environ.get("CF_CONT_ROOT")
    if env:
        return Path(env).expanduser().resolve()
    try:
        here = Path(__file__).resolve()
    except NameError:
        here = Path.cwd().resolve()
    for p in [here] + list(here.parents):
        if p.name == "CF_Continuous":
            return p
        if (p / "data" / "data_20230504").exists() and (p / "codes").exists():
            return p
    return here

PROJ_ROOT = detect_project_root()

# =============================================================
# Paths
# =============================================================
SPLIT_DIR = PROJ_ROOT / "data" / "train_test_split"
DATA_2ND_STAGE_RDS = PROJ_ROOT / "data" / "data_20230504" / "data_2nd_stage.rds"
EVALL_N_SEQ_RDS    = PROJ_ROOT / "data" / "data_20230504" / "evall_N_seq.rds"

# =============================================================
# Output setup (mirrors SO_ANN layout)
# =============================================================
model_tag   = "CF"
n_fields    = 20                 # choose from {1, 3, 5, 10, 20}
run_test_id = 1                  # run only this test_id, like your SO_ANN script

fields_label_map = {1: "one_field", 3: "three_fields", 5: "five_fields",
                    10: "ten_fields", 20: "twenty_fields"}
fields_label = fields_label_map.get(n_fields, f"{n_fields}_fields")

RESULTS_BASE = PROJ_ROOT / "results" / "yield_response_function_for_one_iteration"
RESULTS_DIR  = RESULTS_BASE / f"YRF_{model_tag}_{fields_label}"
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

# =============================================================
# Economic parameters (for true EONR if needed; not used for YRF file)
# =============================================================
p_corn = 6.25 / 25.4  # $/kg
p_N    = 1 / 0.453592 # $/kg

# =============================================================
# Spline basis (mgcv::cr equivalent via patsy)
# =============================================================
from patsy import dmatrix, build_design_matrices, cr
k_splines = 4
spline_formula = f"cr(N, df={k_splines}) - 1"

def prepare_T_mat_py(train_df, test_df):
    """Build train/test spline basis using train support and clip test N to that support."""
    N_train = train_df["N"].to_numpy()
    N_min, N_max = float(np.min(N_train)), float(np.max(N_train))

    T_train_df = dmatrix(spline_formula, {"N": N_train}, return_type="dataframe")
    design_info = T_train_df.design_info

    N_test_clip = np.clip(test_df["N"].to_numpy(), N_min, N_max)
    T_test_mat = build_design_matrices([design_info], {"N": N_test_clip})[0]
    T_test_df = pd.DataFrame(np.asarray(T_test_mat), columns=T_train_df.columns)

    col_names = [f"T_{i+1}" for i in range(T_train_df.shape[1])]
    T_train_df.columns = col_names
    T_test_df.columns  = col_names

    return T_train_df, T_test_df, design_info, (N_min, N_max)

# =============================================================
# Load data
# =============================================================
data_2nd_stage = next(iter(pyreadr.read_r(str(DATA_2ND_STAGE_RDS)).values()))
evall_N_seq    = next(iter(pyreadr.read_r(str(EVALL_N_SEQ_RDS)).values()))

# Ensure numeric types
for c in ['yield', 'y_tilde', 'Nk', 'plateau', 'b0', 'b1', 'b2', 'N']:
    if c in data_2nd_stage.columns:
        data_2nd_stage[c] = pd.to_numeric(data_2nd_stage[c], errors='coerce')
for c in ['sim', 'N', 'N_tilde']:
    if c in evall_N_seq.columns:
        evall_N_seq[c] = pd.to_numeric(evall_N_seq[c], errors='coerce')

# Drop rows with NaNs required for training
data_2nd_stage = data_2nd_stage.dropna(subset=['yield','Nk','plateau','b0','N']).reset_index(drop=True)
evall_N_seq    = evall_N_seq.dropna(subset=['N']).reset_index(drop=True)

# =============================================================
# Load split CSV; restrict to the chosen test_id like SO_ANN
# =============================================================
split_csv_path = SPLIT_DIR / f"train_test_splits_{n_fields}fields.csv"
splits_df = pd.read_csv(split_csv_path)
splits_df = splits_df[splits_df['test_id'] == run_test_id].iloc[:1].copy()

# Features X (environment; same as your CF EONR code)
x_vars = ["Nk", "plateau", "b0"]

# =============================================================
# Main loop (single test_id to mirror SO_ANN example)
# =============================================================
for _, row in tqdm(splits_df.iterrows(), total=len(splits_df), desc="Processing test_id"):
    test_id   = int(row["test_id"])
    train_ids = row[[c for c in row.index if c.startswith("train_")]].dropna().astype(int).tolist()

    # Train/test partitions
    train_df = data_2nd_stage[data_2nd_stage["sim"].isin(train_ids)].copy()
    test_df  = data_2nd_stage[data_2nd_stage["sim"] == test_id].copy()

    # Remove any prior T_ columns
    train_df = train_df.loc[:, ~train_df.columns.str.startswith("T_")]
    test_df  = test_df.loc[:,  ~test_df.columns.str.startswith("T_")]

    # Build spline basis on train support
    T_train_df, T_test_df, design_info, (N_min_tr, N_max_tr) = prepare_T_mat_py(train_df, test_df)
    T_vars   = T_train_df.columns.tolist()

    train_df = pd.concat([train_df.reset_index(drop=True), T_train_df], axis=1)
    test_df  = pd.concat([test_df.reset_index(drop=True),  T_test_df], axis=1)

    # Prepare arrays
    Y     = train_df["yield"].to_numpy()
    X     = train_df[x_vars].to_numpy()
    T_mat = train_df[T_vars].to_numpy()
    W     = X  # same as your CF EONR script
    X_test = test_df[x_vars].to_numpy()

    # Scale X (same as EONR flow)
    X_scaler     = StandardScaler().fit(X)
    X_scaled     = X_scaler.transform(X)
    X_test_scaled= X_scaler.transform(X_test)

    # Train CF
    te_hat_cf = run_CF_c_py(
        Y=Y,
        T=T_mat,
        X=X_scaled,
        W=W,
        n_estimators=2000,
        min_samples_leaf=20,
        random_state=78343
    )

    # Predict per-unit coefficients (one coefficient per spline basis)
    te_hat = te_hat_cf.const_marginal_effect(X_test_scaled)
    te_hat = np.atleast_2d(te_hat)   # shape: [n_test_obs, n_T]
    n_T    = T_mat.shape[1]
    if te_hat.shape[1] != n_T:
        # very defensive; usually matches
        te_hat = np.resize(te_hat, (te_hat.shape[0], n_T))

    # ===== Build evaluation N sequence for THIS sim (mirrors SO_ANN) =====
    eval_seq = evall_N_seq[evall_N_seq['sim'] == test_id].reset_index(drop=True)
    if eval_seq.empty:
        (RESULTS_DIR / f'yield_response_{test_id}_EMPTY_EVAL_SEQ.csv').write_text(
            "No eval N sequence found for this sim\n"
        )
        continue

    Nseq = eval_seq['N'].to_numpy()
    # To avoid spline extrapolation, clip to train N support
    Nseq_clip = np.clip(Nseq, N_min_tr, N_max_tr)

    # Build spline basis for the whole N sequence using the TRAIN design_info
    eval_T_mat  = build_design_matrices([design_info], {"N": Nseq_clip})[0]
    eval_T_full = np.asarray(eval_T_mat)          # shape [L, n_T]
    if eval_T_full.shape[1] != n_T:
        eval_T_full = eval_T_full[:, :n_T]

    # ===== Predicted yield curves: curv[i, :] = te_hat[i] @ eval_T_full.T =====
    curv = te_hat @ eval_T_full.T                 # shape [n_test_obs, L]

    # ===== True yield (Quadratic-Plateau) =====
    req_true = ['b0','b1','b2','Nk']
    missing = [c for c in req_true if c not in test_df.columns]
    if missing:
        raise ValueError(f"Missing columns for true yield (QP): {missing}")

    b0_vec = test_df['b0'].to_numpy()
    b1_vec = test_df['b1'].to_numpy()
    b2_vec = test_df['b2'].to_numpy()
    Nk_vec = test_df['Nk'].to_numpy()

    # Broadcast to produce [n_test_obs, L]
    quad   = b0_vec[:, None] + b1_vec[:, None]*Nseq[None, :] + b2_vec[:, None]*(Nseq[None, :]**2)
    plate  = b0_vec[:, None] + b1_vec[:, None]*Nk_vec[:, None] + b2_vec[:, None]*(Nk_vec[:, None]**2)
    true_y = np.where(Nseq[None, :] < Nk_vec[:, None], quad, plate)

    # ===== Assemble output like SO_ANN =====
    id_cols = [c for c in ['aunit_id', 'cell_id', 'field_id'] if c in test_df.columns]
    L = len(Nseq)
    rows = []
    for i in range(test_df.shape[0]):
        row_dict = {
            'sim':        [test_id] * L,
            'row_id':     [i] * L,
            'N':          Nseq,
            'pred_yield': curv[i, :],
            'true_yield': true_y[i, :]
        }
        if 'N_tilde' in eval_seq.columns:
            row_dict['N_tilde'] = eval_seq['N_tilde'].to_numpy()
        for col in id_cols:
            row_dict[col] = [test_df.loc[i, col]] * L
        rows.append(pd.DataFrame(row_dict))

    df_out = pd.concat(rows, ignore_index=True)
    out_path = RESULTS_DIR / f'yield_response_{test_id}.csv'
    df_out.to_csv(out_path, index=False)
    print(f"✅ Wrote {out_path}")


NameError: name '__file__' is not defined